# SYLAP control 

##### The next cell it will load class Sylap and methods

In [15]:
import serial, time

class Sylap:
    def __init__(self, mySerialPort='/dev/ttyUSB0', sysClk_MHz=400., clkRef_ICTRL_MHz=400.,verbose=False):
        print("Try to connect to ",mySerialPort)
        self.ser = serial.Serial(mySerialPort,baudrate=115200, bytesize=8, parity='N')
        print("Connected to ",self.ser.name)
        self.sysClk_MHz=sysClk_MHz
        self.clkRef_ICTRL_MHz=clkRef_ICTRL_MHz        
        self.sysClk_ps=1000./self.sysClk_MHz
        self.IDELAY_ps=1000000./self.clkRef_ICTRL_MHz/64.
        self.laser_MHz=0.
        self.pulseLength=0.
        self.pulseLength_ps=0.*self.sysClk_ps
        self.maxTotalOffset=0.
        self.currentTotalOffset=0
        print("System clock: ", self.sysClk_MHz, "MHz = ",self.sysClk_ps, "ps") 
        print("IDELAY clock: ", self.clkRef_ICTRL_MHz, "MHz\nOffset granularity: ",self.IDELAY_ps, "ps\n") 
        print("Remeber to enable the output with .setEnable() methods or call .typicalConfiguration()")
        self.verbose=verbose
    def typicalConfiguration(self):
        self.setLaserLength(100)
        self.setPulseLength(1)
        self.setTotalOffset(0)
        self.setEnable()
    def sendCommand(self,payload):
        toSend=[0xAA,0xBB,0xCC]+payload
        if self.verbose:
            print([hex(i) for i in toSend])
            
            
        self.ser.write(toSend)
    def setReg3byte(self,data,reg):
        self.sendCommand([0x00 | reg,
                  (data & 0xFF0000) >> 16 ,
                  (data & 0x00FF00) >> 8,
                  (data & 0x0000FF ),
                   0x80 | reg ])
    def setEnable(self):
        self.setReg3byte(1,0)
    def setDisable(self):
        self.setReg3byte(0,0)
    def setLaserLength(self,numberOfClocks):
        if ( numberOfClocks % 2  == 1):
            print ("numberOfClocks is not even, we approximating to ", int(numberOfClocks/2)*2 )
        return self.setLaserHalfLength(int(numberOfClocks/2))

    def setTotalOffsetCoarse(self,delayInSysClk):
        self.setReg3byte(delayInSysClk,1)
    def setPulseLength(self,numberOfSysClocks):
        self.pulseLength=numberOfSysClocks
        self.pulseLength_ps=numberOfSysClocks*self.sysClk_ps
        self.setReg3byte(numberOfSysClocks,2)
    def setLaserHalfLength(self,laserHalfLength):
        self.laser_MHz=400/(2*(laserHalfLength+1))
        self.setReg3byte(laserHalfLength,3)   
        self.maxTotalOffset=((laserHalfLength)*2)*64-1
    def setLaserMaxCount(self,lengthLaserClk):
        self.setReg3byte(lengthLaserClk,4)
        
    def setTotalOffset(self,delay):
        self.setReg3byte(int(delay)<<2,0x11)
        self.currentTotalOffset=delay
        self.currentTotalOffset_ps=delay*self.sysClk_ps/64.
        
    def setTotalOffsetInPs(self,delay):
        self.setReg3byte(int(delay/self.IDELAY_ps)<<2,0x11)
        return int(delay/self.IDELAY_ps)*self.IDELAY_ps

    def setReset(self):
        self.setReg3byte(0,0x10)
    
    def setDummy(self): #not existing reg
        self.setReg3byte(0,0x12)        

##### Here Sylap object is created, set to verbose and a Reset command is send to the device

In [14]:
s=Sylap('/dev/ttyUSB0')
s.verbose=True
s.setReset()

Try to connect to  /dev/ttyUSB0


SerialException: [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'

In [13]:
s.setEnable()
s.setDummy()

NameError: name 's' is not defined

0xCC##### Set a configuration and output enable

In [7]:
s.verbose=True
s.setPulseLength(3)
print(s.sysClk_ps*7)
s.setLaserMaxCount(5)
s.setLaserHalfLength(4)
s.setTotalOffset(s.maxTotalOffset)


print ("Laser =",s.laser_MHz," MHz")
print ("Pulse Length =",s.pulseLength_ps," ps")
print ("Current offset = ",s.currentTotalOffset, " = ", s.currentTotalOffset_ps," ps")
print ("Max offset = ",s.maxTotalOffset, " = ", s.maxTotalOffset*s.pulseLength_ps," ps")

NameError: name 's' is not defined

In [19]:
s.verbose=False

print("Sweep start")
for i in range(0,s.maxTotalOffset):
    s.setTotalOffset(i)
    time.sleep(0.01)
print("Sweep end")

Sweep start


KeyboardInterrupt: 